# ngrams

> Functionality for ngram analysis.

In [ ]:
#| default_exp ngrams

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import numpy as np
import time
import polars as pl
from spacy.attrs import ORTH, LOWER # remove? - add ENT_TYPE, ENT_IOB
import math
from fastcore.basics import patch

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE, set_logger_state

In [ ]:
#| hide
import os

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

path_to_toy_corpus = f'{save_path}toy.corpus'
path_to_brown_corpus = f'{save_path}brown.corpus'
path_to_reuters_corpus = f'{save_path}reuters.corpus'
path_to_congress_corpus = f'{save_path}us-congressional-speeches-subset-100k.corpus'

In [ ]:
# #| hide
# for path in [path_to_toy_corpus, path_to_brown_corpus, path_to_reuters_corpus, path_to_congress_corpus]:
#     corpus = Corpus().load(path)
#     df = corpus.spaces.collect()
#     df = df.with_columns(
#         pl.col('position').shift(-1).alias('position2')
#     )
#     display(df.filter(pl.col('position') + 1 == pl.col('position2')).head())

In [ ]:
#| export
class Ngrams:
	""" Class for n-gram analysis reporting. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| hide
toy = Corpus().load(path_to_toy_corpus)
ngrams_toy = Ngrams(toy)

In [ ]:
#| exporti
@patch
def _get_ngrams(self:Ngrams, 
			   token_sequence: list[np.ndarray], # token sequence to get index for 
			   index_id: int, # index to search (i.e. ORTH, LOWER)
			   token_positions: list[np.ndarray], # positions of token sequence, returned by get_token_positions 
			   ngram_length: int = 2, # length of ngram
			   ngram_token_position: str = 'LEFT', # specify if token sequence is on LEFT, RIGHT, or MIDDLE of ngrams
			   exclude_punctuation:bool=False, # exclude punctuation tokens
			   exclude_spaces:bool=False # exclude space tokens
			   ) -> np.ndarray: # array of ngrams results

	""" Get ngram data for a token sequence. """
	
	start_time = time.time()
	sequence_len = len(token_sequence[0])
	variants_len = len(token_sequence)
	token_index_len = len(token_positions[0])

	if index_id == ORTH:
		index = 'orth_index'
	else:
		index = 'lower_index'

	if ngram_token_position == 'RIGHT':
		ngram_range = range(-1 * ngram_length + sequence_len, sequence_len)
	elif ngram_token_position == 'MIDDLE':
		ngram_range = range(-1 * ngram_length + sequence_len + 1, sequence_len + 1)
	else:
		ngram_range = range(0, ngram_length)

	ngrams = []
	
	for pos in ngram_range:
		if variants_len == 1 and pos > -1 and pos < sequence_len:
			ngrams.append(np.full(token_index_len, token_sequence[0][pos]))
		else:
			seq = token_positions[0] + pos
			ngrams.append(self.corpus.get_tokens_by_index(index)[seq])

	ngrams = np.stack(ngrams)

	# getting positions to search for EOF_TOKEN and filter out ngrams crossing doc boundaries
	positions = (np.array(ngram_range)[:, None] != np.arange(sequence_len)).all(axis=1)
	ngrams = np.delete(ngrams, np.where(ngrams[positions] == self.corpus.EOF_TOKEN)[1], axis=1)

	if exclude_punctuation:
		punctuation_tokens = self.corpus.punct_tokens
		ngrams = np.delete(ngrams, np.where(np.isin(ngrams, punctuation_tokens))[1], axis=1)

	logger.info(f'Ngrams ({ngrams.shape[1]}) retrieval time: {(time.time() - start_time):.5f} seconds')
	return ngrams


In [ ]:
#| hide
token_str = 'dog'
token_sequence, index_id = toy.tokenize(token_str, simple_indexing=True)
token_positions = toy.get_token_positions(token_sequence, index_id)
ngrams = ngrams_toy._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 2, ngram_token_position = 'LEFT')
assert tuple(toy.token_ids_to_tokens(ngrams[0])) == tuple(['dog', 'dog', 'dog'])
assert tuple(toy.token_ids_to_tokens(ngrams[1])) == tuple(['sat', 'is', 'is'])

2025-06-08 15:11:24 - INFO - tokenize - Tokenization time: 0.00031 seconds
2025-06-08 15:11:24 - INFO - get_token_positions - Token indexing (3) time: 0.00021 seconds
2025-06-08 15:11:24 - INFO - _get_ngrams - Ngrams (3) retrieval time: 0.00038 seconds


In [ ]:
#| hide
brown = Corpus().load(path_to_brown_corpus)
ngrams_brown = Ngrams(brown)
token_str = 'The'
token_sequence, index_id = brown.tokenize(token_str, simple_indexing=True)
token_positions = brown.get_token_positions(token_sequence, index_id)
ngrams = ngrams_brown._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 3, ngram_token_position = 'RIGHT', exclude_punctuation = True)
print(ngrams.shape)
# find any ngrams[0] that contain brown.space_tokens
from_x = 25
to_x = 30
print(brown.token_ids_to_tokens(ngrams[0][from_x:to_x]))
print(brown.token_ids_to_tokens(ngrams[1][from_x:to_x]))
print(brown.token_ids_to_tokens(ngrams[2][from_x:to_x]))

2025-06-08 15:26:13 - INFO - memory_usage - init, memory usage: 1446.75390625 MB


2025-06-08 15:26:14 - INFO - load - Load time: 0.882 seconds
2025-06-08 15:26:14 - INFO - _init_token_arrays - Created tokens_array in 0.022 seconds
2025-06-08 15:26:14 - INFO - _init_token_arrays - Created tokens_lookup in 0.027 seconds
2025-06-08 15:26:14 - INFO - _init_token_arrays - Created tokens_sort_order in 0.086 seconds
2025-06-08 15:26:14 - INFO - tokenize - Tokenization time: 0.13876 seconds
2025-06-08 15:26:14 - INFO - get_token_positions - Token indexing (63516) time: 0.00930 seconds
2025-06-08 15:26:14 - INFO - _get_ngrams - Ngrams (49439) retrieval time: 0.01026 seconds


(3, 49439)
['protected' 'it' 'operating' 'in' 'that']
['\n' 'found' 'procedures' 'effect' 'when']
['the' 'the' 'the' 'the' 'the']


In [ ]:
# load the corpus
reuters = Corpus().load(path_to_reuters_corpus)

In [ ]:
# instantiate the Ngrams class
ngrams_reuters = Ngrams(reuters)

In [ ]:
#| hide
token_str = 'ocean'
token_sequence, index_id = reuters.tokenize(token_str, simple_indexing=True)
token_positions = reuters.get_token_positions(token_sequence, index_id)
ngrams = ngrams_reuters._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 2, ngram_token_position = 'LEFT')
print(ngrams)

[[48119 48119 48119 48119 48119 48119 48119 48119 48119 48119 48119 48119
  48119 48119 48119 48119 48119 48119 48119 48119 48119 48119 48119 48119
  48119 48119 48119 48119 48119]
 [13571 52710 46667 60529 46667 60529   936 71188 32924 72663 32924 72663
   9490 43372 26836 12053 53521 11720 13571 11720 11720 52211 13571 40736
  10743 67894 12439 65912 75228]]


In [ ]:
#| export
@patch
def ngrams(self: Ngrams, 
		   token_str: str, # token string to get ngrams for 
		   ngram_length:int = 2, # length of ngram
		   ngram_token_position:str = 'LEFT', # specify if token sequence is on LEFT, RIGHT, or MIDDLE of ngrams
		   normalize_by:int=10000, # normalize frequencies by a number (e.g. 10000)
		   page_size:int = PAGE_SIZE, # number of results to display per results page 
		   page_current:int = 1, # current page of results
		   show_all_columns:bool = False, # return raw df with all columns or just ngram and frequency
		   exclude_punctuation:bool=True, # exclude punctuation tokens
		   exclude_spaces:bool=True, # exclude space tokens
		   use_cache:bool = True # retrieve the results from cache if available
		   ) -> Result: # return a Result object with ngram data
	""" Report ngrams for a token string. """

	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)

	start_time = time.time()
	cache_id = tuple(['ngram'] + list(token_sequence) + [ngram_length, ngram_token_position])

	if use_cache == True and cache_id in self.corpus.results_cache:
		logger.info('Using cached ngrams results')
		ngrams_report = self.corpus.results_cache[cache_id][0]
		total_unique = self.corpus.results_cache[cache_id][1]
		total_count = self.corpus.results_cache[cache_id][2]
	else:
		token_positions = self.corpus.get_token_positions(token_sequence, index_id)
		
		if len(token_positions[0]) == 0:
			logger.info('No tokens found')
			return Result(type = 'ngrams', df=pl.DataFrame(), title=f'Ngrams for "{token_str}"', description=f'No matches', summary_data={}, formatted_data=[])

		logger.info('Generating ngrams results')
		ngrams = self._get_ngrams(token_sequence, index_id, token_positions, ngram_length = ngram_length, ngram_token_position = ngram_token_position)
		total_count = ngrams.shape[1]
		schema = [f'token_{i+1}' for i in range(ngram_length)]
		ngrams_report = pl.DataFrame(ngrams.T, schema=schema).to_struct(name = 'ngram_token_ids').value_counts(sort=True).rename({"count": "frequency"})
		ngrams_report = ngrams_report.with_row_index(name='rank', offset=1)
		total_unique = len(ngrams_report)
		self.corpus.results_cache[cache_id] = (ngrams_report, total_unique, total_count)
	
	count_tokens, tokens_descriptor, total_descriptor = self.corpus.get_token_count_text(exclude_punctuation, exclude_spaces)

	resultset_start = page_size*(page_current-1)

	# get specific chunk of report into polars based on resultset_start:
	ngrams_report_page = ngrams_report.slice(resultset_start, page_size).unnest('ngram_token_ids')
	ngrams_report_page = ngrams_report_page.with_columns(((pl.col("frequency") / pl.lit(count_tokens)) * normalize_by).alias('normalized_frequency'))

	token_strs = []
	for i in range(ngram_length):
		token_strs.append(self.corpus.token_ids_to_tokens(ngrams_report_page[f'token_{i+1}'].to_numpy()))
	token_strs = np.array(token_strs)
	ngram_text = [' '.join(column) for column in token_strs.T]
	ngrams_report_page = ngrams_report_page.with_columns(pl.Series(name="ngram", values=ngram_text))
	#ngrams_report_page = ngrams_report_page.to_pandas().set_index('rank')
	total_pages = math.ceil(total_unique/page_size)
	summary_data = {'ngram_length': ngram_length, 'ngram_token_position': ngram_token_position, 'total_unique': total_unique, 'total_count': total_count, 'page_current': page_current, 'total_pages': total_pages}
	formatted_data = [f'Report based on {tokens_descriptor}', f'Ngram length: {ngram_length}, Token position: {ngram_token_position.lower()}']

	if normalize_by is not None:
		formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')

	formatted_data.extend([f'Total unique ngrams: {total_unique:,}', f'Total ngrams: {total_count:,}'])

	if page_size != 0 and total_count > page_size:
		formatted_data.extend([f'Showing {min(page_size, total_count)} rows', f'Page {page_current} of {total_pages}']) 

	if show_all_columns == False:
		ngrams_report_page = ngrams_report_page[['rank', 'ngram', 'frequency', 'normalized_frequency']]
	
	logger.info(f'Ngrams report time: {(time.time() - start_time):.5f} seconds')

	return Result(type = 'ngrams', df=ngrams_report_page, title=f'Ngrams for "{token_str}"', description=f'{self.corpus.name}', summary_data=summary_data, formatted_data=formatted_data)


In [ ]:
# run the ngrams method and display the results
ngrams_reuters.ngrams('environmental', ngram_length = 2, ngram_token_position = 'LEFT').display()

Ngrams for "environmental" 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 environmental systems 
 4 
 0.03 
 
 
 2 
 environmental 
 
 4 
 0.03 
 
 
 3 
 environmental protection 
 3 
 0.02 
 
 
 4 
 environmental damage 
 2 
 0.01 
 
 
 5 
 environmental , 
 2 
 0.01 
 
 
 6 
 environmental impact 
 2 
 0.01 
 
 
 7 
 environmental controls 
 1 
 0.01 
 
 
 8 
 environmental approval 
 1 
 0.01 
 
 
 9 
 environmental and 
 1 
 0.01 
 
 
 10 
 environmental sciences 
 1 
 0.01 
 
 
 11 
 environmental service 
 1 
 0.01 
 
 
 12 
 environmental concerns 
 1 
 0.01 
 
 
 13 
 environmental regulations 
 1 
 0.01 
 
 
 14 
 environmental issues 
 1 
 0.01 
 
 
 15 
 environmental services 
 1 
 0.01 
 
 
 16 
 environmental power 
 1 
 0.01 
 
 
 17 
 environmental plan 
 1 
 0.01 
 
 
 18 
 environmental had 
 1 
 0.01 
 
 
 19 
 environmental control 
 1 
 0.01 
 
 
 20 
 environmental management 
 1 
 0.01 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 2, Token position: left 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 23 
 


 
 Total ngrams: 34 
 


 
 Showing 20 rows 
 


 
 Page 1 of 2

In [ ]:
# run the ngrams method and display the results
ngrams_reuters.ngrams('the highest', ngram_length = 3, ngram_token_position = 'LEFT', page_size = 10).display()

Ngrams for "the highest" 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the highest 
 
 8 
 0.06 
 
 
 2 
 the highest since 
 5 
 0.04 
 
 
 3 
 the highest in 
 3 
 0.02 
 
 
 4 
 the highest level 
 2 
 0.01 
 
 
 5 
 the highest such 
 2 
 0.01 
 
 
 6 
 the highest positive 
 2 
 0.01 
 
 
 7 
 the highest depreciation 
 1 
 0.01 
 
 
 8 
 the highest rises 
 1 
 0.01 
 
 
 9 
 the highest set 
 1 
 0.01 
 
 
 10 
 the highest daily 
 1 
 0.01 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3, Token position: left 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 26 
 


 
 Total ngrams: 42 
 


 
 Showing 10 rows 
 


 
 Page 1 of 3

In [ ]:
#| export
@patch
def ngram_frequencies(self: Ngrams, 
                ngram_length:int=2, # length of ngram
                case_sensitive:bool=False, # frequencies for tokens lowercased or with case preserved
				normalize_by:int=10000, # normalize frequencies by a number (e.g. 10000)
				page_size:int=PAGE_SIZE, # number of rows to return
				page_current:int=1, # current page
				exclude_punctuation:bool=True, # exclude punctuation tokens
				exclude_spaces:bool=True # exclude space tokens
				) -> Result: # return a Result object with the frequency table
	""" Report frequent ngrams. """
	
	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	start_time = time.time()

	if case_sensitive:
		index = 'orth_index'
	else:
		index = 'lower_index'

	filter = [self.corpus.EOF_TOKEN]
	if exclude_punctuation == True:
		filter += self.corpus.punct_tokens
	if exclude_spaces == True:
		filter += self.corpus.space_tokens

	resultset_start = page_size*(page_current-1)

	count_tokens, tokens_descriptor, total_descriptor = self.corpus.get_token_count_text(exclude_punctuation, exclude_spaces)
	formatted_data = [f'Report based on {tokens_descriptor}']
	formatted_data.append(f'Ngram length: {ngram_length}')

	# ngrams = self.corpus.get_ngrams_by_index(ngram_length = ngram_length, index = index).T
	ngrams_report = self.corpus.tokens.with_columns([pl.col(index).shift(-i).alias(f'token_{i+1}') for i in range(ngram_length)])

	schema = [f'token_{i+1}' for i in range(ngram_length)]

	# ngrams_report = pl.LazyFrame(ngrams.T, schema=schema)
	ngrams_report = ngrams_report.group_by(schema).agg(pl.len().alias("frequency")).sort(by="frequency", descending=True)

	for i in range(ngram_length):
		ngrams_report = ngrams_report.filter(~pl.col(f'token_{i+1}').is_in(filter))

	ngrams_report_page = ngrams_report.slice(resultset_start, page_size).collect(engine = 'streaming')
	logger.info(f'collected report page: {(time.time() - start_time):.5f} seconds')
	token_strs = []
	for i in range(ngram_length):
		token_strs.append(self.corpus.token_ids_to_tokens(ngrams_report_page.select(pl.col(f'token_{i+1}')).to_numpy().flatten()))
	token_strs = np.array(token_strs)
	ngram_text = [' '.join(column) for column in token_strs.T]
	ngrams_report_page = ngrams_report_page.with_columns(pl.Series(name="ngram", values=ngram_text)).with_row_index(name='rank', offset=(page_current-1)*page_size+1)
	ngrams_report_page = ngrams_report_page.with_columns(((pl.col("frequency") / pl.lit(count_tokens)) * normalize_by).alias('normalized_frequency'))
	formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')

	# formatted_data.extend([f'Total unique ngrams: {total_unique:,}', f'Total ngrams: {total_count:,}'])

	# total_pages = math.ceil(total_unique/page_size)
	# if page_size != 0 and total_count > page_size:
	# 	formatted_data.extend([f'Showing {min(page_size, total_count)} rows', f'Page {page_current} of {total_pages}']) 

	ngrams_report_page = ngrams_report_page[['rank', 'ngram', 'frequency', 'normalized_frequency']]

	return Result(type = 'ngrams', df=ngrams_report_page, title=f'Ngram Frequencies', description=f'{self.corpus.name}', summary_data = {}, formatted_data = formatted_data)

In [ ]:
set_logger_state('verbose')

In [ ]:
ngrams_reuters.ngram_frequencies(ngram_length = 3, case_sensitive = True).display()

2025-06-08 14:42:30 - INFO - ngram_frequencies - collected report page: 0.15831 seconds


Ngram Frequencies 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 The company said 
 741 
 5.30 
 
 
 2 
 cts vs loss 
 644 
 4.60 
 
 
 3 
 mln dlrs in 
 624 
 4.46 
 
 
 4 
 pct of the 
 539 
 3.85 
 
 
 5 
 said it has 
 529 
 3.78 
 
 
 6 
 the end of 
 444 
 3.17 
 
 
 7 
 dlrs a share 
 441 
 3.15 
 
 
 8 
 cts a share 
 430 
 3.07 
 
 
 9 
 the United States 
 418 
 2.99 
 
 
 10 
 Inc said it 
 412 
 2.95 
 
 
 11 
 the company 's 
 408 
 2.92 
 
 
 12 
 said in a 
 386 
 2.76 
 
 
 13 
 is expected to 
 358 
 2.56 
 
 
 14 
 the company said 
 351 
 2.51 
 
 
 15 
 cts vs profit 
 345 
 2.47 
 
 
 16 
 Corp said it 
 342 
 2.44 
 
 
 17 
 It said the 
 312 
 2.23 
 
 
 18 
 said it will 
 311 
 2.22 
 
 
 19 
 billion dlrs in 
 307 
 2.19 
 
 
 20 
 He said the 
 295 
 2.11 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3 
 


 
 Normalized Frequency is per 10,000 tokens

In [ ]:
#| hide
congress = Corpus().load(f'{save_path}us-congressional-speeches-subset-100k.corpus')
report_congress = Ngrams(congress)
congress._init_token_arrays()

2025-06-08 14:42:30 - INFO - memory_usage - init, memory usage: 490.1640625 MB
2025-06-08 14:42:31 - INFO - load - Load time: 0.663 seconds
2025-06-08 14:42:31 - INFO - _init_token_arrays - Created tokens_array in 0.123 seconds
2025-06-08 14:42:31 - INFO - _init_token_arrays - Created tokens_lookup in 0.090 seconds
2025-06-08 14:42:32 - INFO - _init_token_arrays - Created tokens_sort_order in 0.926 seconds


In [ ]:
#| hide
%time report_congress.ngram_frequencies(ngram_length = 2, case_sensitive = False, page_current = 1).display()

2025-06-08 14:42:33 - INFO - ngram_frequencies - collected report page: 1.34824 seconds


Ngram Frequencies 
 
 
 US Congressional Speeches Subset 100k 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 of the 
 227,943 
 126.49 
 
 
 2 
 in the 
 114,241 
 63.39 
 
 
 3 
 to the 
 92,967 
 51.59 
 
 
 4 
 it is 
 51,659 
 28.67 
 
 
 5 
 that the 
 51,620 
 28.64 
 
 
 6 
 for the 
 46,516 
 25.81 
 
 
 7 
 on the 
 43,924 
 24.37 
 
 
 8 
 and the 
 43,053 
 23.89 
 
 
 9 
 by the 
 40,236 
 22.33 
 
 
 10 
 the senator 
 37,269 
 20.68 
 
 
 11 
 to be 
 34,012 
 18.87 
 
 
 12 
 senator from 
 30,640 
 17.00 
 
 
 13 
 the gentleman 
 30,142 
 16.73 
 
 
 14 
 i am 
 29,952 
 16.62 
 
 
 15 
 with the 
 28,558 
 15.85 
 
 
 16 
 mr. speaker 
 27,877 
 15.47 
 
 
 17 
 mr. president 
 27,233 
 15.11 
 
 
 18 
 of this 
 26,957 
 14.96 
 
 
 19 
 the senate 
 26,397 
 14.65 
 
 
 20 
 the committee 
 26,217 
 14.55 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 2 
 


 
 Normalized Frequency is per 10,000 tokens

CPU times: user 7.7 s, sys: 626 ms, total: 8.32 s
Wall time: 1.37 s


In [ ]:
#| hide
%time report_congress.ngrams('liberty', ngram_length = 2, ngram_token_position = 'RIGHT', page_size = 10).display()

2025-06-08 14:42:33 - INFO - tokenize - Tokenization time: 0.00036 seconds
2025-06-08 14:42:34 - INFO - get_token_positions - Token indexing (1060) time: 0.55777 seconds
2025-06-08 14:42:34 - INFO - ngrams - Generating ngrams results
2025-06-08 14:42:34 - INFO - _get_ngrams - Ngrams (1060) retrieval time: 0.00192 seconds
2025-06-08 14:42:34 - INFO - ngrams - Ngrams report time: 0.56840 seconds


Ngrams for "liberty" 
 
 
 US Congressional Speeches Subset 100k 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 of liberty 
 227 
 0.13 
 
 
 2 
 the liberty 
 124 
 0.07 
 
 
 3 
 . liberty 
 99 
 0.05 
 
 
 4 
 at liberty 
 98 
 0.05 
 
 
 5 
 and liberty 
 62 
 0.03 
 
 
 6 
 for liberty 
 41 
 0.02 
 
 
 7 
 religious liberty 
 28 
 0.02 
 
 
 8 
 individual liberty 
 28 
 0.02 
 
 
 9 
 their liberty 
 18 
 0.01 
 
 
 10 
 radio liberty 
 15 
 0.01 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 2, Token position: right 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 156 
 


 
 Total ngrams: 1,060 
 


 
 Showing 10 rows 
 


 
 Page 1 of 16

CPU times: user 443 ms, sys: 340 ms, total: 783 ms
Wall time: 589 ms


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()